In [1]:
import sys, os
from tqdm import tqdm
sys.path.append(R"D:\GITHUB\FunAudioLLM-APP\cosyvoice")
sys.path.append(R"D:\GITHUB\FunAudioLLM-APP\cosyvoice\third_party\Matcha-TTS")

from cosyvoice.cli.cosyvoice import CosyVoice, CosyVoice2
from cosyvoice.utils.file_utils import load_wav
from cosyvoice.utils.common import set_all_random_seed
import torchaudio

2025-02-19 20:23:30,501 - modelscope - INFO - PyTorch version 2.3.1+cu121 Found.
2025-02-19 20:23:30,504 - modelscope - INFO - Loading ast index from C:\Users\CHENHAOYUAN\.cache\modelscope\ast_indexer
2025-02-19 20:23:30,702 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 553a05bea23a95025342c8774cbd7108 and a total number of 980 components indexed


failed to import ttsfrd, use WeTextProcessing instead


## 加载模型

In [2]:
cosyvoice = CosyVoice2(R'D:\GITHUB\FunAudioLLM-APP\cosyvoice\pretrained_models\CosyVoice2-0.5B', load_jit=False, load_trt=False, fp16=False)

d:\anaconda3\envs\cosyvoice\lib\site-packages\diffusers\models\lora.py:393: FutureWarning: `LoRACompatibleLinear` is deprecated and will be removed in version 1.0.0. Use of `LoRACompatibleLinear` is deprecated. Please switch to PEFT backend by installing PEFT: `pip install peft`.
  deprecate("LoRACompatibleLinear", "1.0.0", deprecation_message)
2025-02-19 20:23:41,764 INFO input frame rate=25
d:\anaconda3\envs\cosyvoice\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(
2025-02-19 20:23:43,577 WETEXT INFO building fst for zh_normalizer ...
2025-02-19 20:23:43,577 INFO building fst for zh_normalizer ...
2025-02-19 20:24:07,856 WETEXT INFO done
2025-02-19 20:24:07,856 INFO done
2025-02-19 20:24:07,857 WETEXT INFO fst path: d:\anaconda3\envs\cosyvoice\lib\site-packages\tn\zh_tn_tagger.fst
202

## 加载音频
需要手动标记一下音频的内容

In [3]:
prompt_audio = load_wav(R'D:\GITHUB\OpenVoice\inputs\train\tvboy1_denoised.wav', 16000)
prompt_text = "两个或者三个吧，嗯肚子很胀，就是，有点，嗯，有时候会吐的"

## 配置

In [4]:
batch = 4 # doesn't work at this moment
output_dir = "./assets/dataset/tvboy_denoised_mandarin_seed34751218"
seed = 34751218

# 以下选一个

## 普通话
input_txts = "./assets/text/mandarin.txt"
cross_lang = ""

## 粤语
# input_txts = "./cantonese.txt"
# cross_lang = "粤语"

# ！！下面的不要动

if not os.path.exists(output_dir):
	os.makedirs(output_dir, exist_ok=True)
with open(input_txts, 'r', encoding='utf-8') as f:
	input_text_lines = [ x.strip() for x in f.readlines() if x.strip() != "" ]
	print("Got dataset: ", len(input_text_lines))
	def input_generator():
		for i in range(0, len(input_text_lines), batch):
			yield input_text_lines[i:min(i+batch, len(input_text_lines))]
set_all_random_seed(seed)
print("Output folder: ", output_dir)
print("Using seed: ", seed)

Got dataset:  729
Output folder:  ./assets/dataset/tvboy_denoised_mandarin_seed34751218
Using seed:  34751218


## 测试推理

In [61]:
sample_text = input_text_lines[0]
def zero_infer():
	print("Using zero infer.")
	ret = list(cosyvoice.inference_zero_shot(sample_text, prompt_text, prompt_audio, stream=False))[0]
	torchaudio.save("zero_infer.wav", ret['tts_speech'], cosyvoice.sample_rate)
def cross_infer():
	print("Using cross shot to " + cross_lang + ".")
	instruct_text = f"用{cross_lang}说这句话"
	ret = list(cosyvoice.inference_instruct2(sample_text, instruct_text, prompt_audio, stream=False))[0]
	torchaudio.save("cross_infer.wav", ret['tts_speech'], cosyvoice.sample_rate)
if cross_lang == "":
	zero_infer()
else:
	cross_infer()
print("Done.")

Using zero infer.


  0%|          | 0/1 [00:00<?, ?it/s]2025-02-17 11:49:48,140 INFO synthesis text 他在家里吃苹果，喝茶，打电话，唱歌。
2025-02-17 11:49:53,008 INFO yield speech len 6.92, rtf 0.7034620453167513
100%|██████████| 1/1 [00:05<00:00,  5.28s/it]

Done.


## 生成

In [5]:
def zero_shot():
	print("Using zero shot.")
	for idx, input_data in enumerate(tqdm(input_generator())):
		for subidx, data in enumerate(input_data):
			global_idx = idx * batch + subidx + 1
			filename = os.path.join(output_dir, f"{global_idx}.wav")
			if os.path.exists(filename):
				print("Skip ", filename)
				continue
			ret = list(cosyvoice.inference_zero_shot(data, prompt_text, prompt_audio, stream=False))[0]
			torchaudio.save(filename, ret['tts_speech'], cosyvoice.sample_rate)
def cross_shot():
	print("Using cross shot to " + cross_lang + ".")
	instruct_text = f"用{cross_lang}说这句话"
	for idx, input_data in enumerate(tqdm(input_generator())):
		for subidx, data in enumerate(input_data):
			global_idx = idx * batch + subidx + 1
			filename = os.path.join(output_dir, f"{global_idx}.wav")
			if os.path.exists(filename):
				print("Skip ", filename)
				continue
			ret = list(cosyvoice.inference_instruct2(data, instruct_text, prompt_audio, stream=False))[0]
			torchaudio.save(filename, ret['tts_speech'], cosyvoice.sample_rate)
if cross_lang == "":
	zero_shot()
else:
	cross_shot()
print("Done.")

Using zero shot.


0it [00:00, ?it/s]

Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\1.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\2.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\3.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\4.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\5.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\6.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\7.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\8.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\9.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\10.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\11.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\12.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\13.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\14.wav
Skip  ./assets/dataset/tvboy_denoised_mandarin_seed34751218\15.wav
Skip

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-19 20:24:41,311 INFO synthesis text 今天的天气真好，我们一起去公园散步吧！
d:\anaconda3\envs\cosyvoice\lib\site-packages\transformers\models\qwen2\modeling_qwen2.py:544: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)
2025-02-19 20:24:46,015 INFO yield speech len 4.52, rtf 1.0407022670306991
100%|██████████| 1/1 [00:05<00:00,  5.39s/it]
2025-02-19 20:24:46,034 WARNING synthesis text 我喜欢吃苹果和香蕉。 too short than prompt text 两个或者三个吧，嗯肚子很胀，就是，有点，嗯，有时候会吐的, this may lead to bad performance


...

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-19 20:58:20,517 INFO synthesis text 在这个信息时代，技术的发展日新月异，许多新的科技产品层出不穷，给我们的生活带来了极大的便利，改变了我们工作、学习和娱乐的方式。
2025-02-19 20:58:30,699 INFO yield speech len 14.6, rtf 0.6973788183029384
100%|██████████| 1/1 [00:10<00:00, 10.76s/it]
182it [33:50, 43.94s/it]

min value is  tensor(-1.0223)
max value is  tensor(1.0314)


2025-02-19 20:58:31,302 INFO synthesis text 这场演唱会的每一首歌都深深打动了观众，无论是经典的老歌，还是新推出的作品，歌手们的精湛演技和感人演绎都让观众久久不能平静，充满感动。
2025-02-19 20:58:40,932 INFO yield speech len 14.0, rtf 0.6877973079681396
100%|██████████| 1/1 [00:10<00:00, 10.21s/it]
183it [34:00, 11.15s/it]

Done.
